<a href="https://colab.research.google.com/github/tg-bomze/ENTAR/blob/master/ENTAR_Rus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <b><font color="black" size="-5">.</font></b>
![alt text](https://raw.githubusercontent.com/tg-bomze/ENTAR/master/entar.png)

<b><font color="black" size="+3">Ансамбль Нейронных Инструментов для Реставрации Aнимаций</font></b>

*↓ Открой меня ↓*

##**Краткая инструкция:**

Для запуска нажмите **Open in playground** в верхнем левом углу:

![alt text](https://d.radikal.ru/d39/2001/c1/4192dfd3a667.jpg)

Далее, нажав на квадратные скобки выполняйте каждый блок поочередно, дожидаясь окончания выполнения предыдущего:

![alt text](https://d.radikal.ru/d18/2001/1a/b618f3778f86.jpg)

*Блок считается выполненным, когда вокруг квадратных скобок* **[_]** *перестанет бегать "змейка"*

##**Собрал все воедино:**
GitHub: [@tg-bomze](https://github.com/tg-bomze)

Telegram: [@bomze](https://t.me/bomze) 

Twitter: [@tg_bomze](https://twitter.com/tg_bomze)

*По всем вопросам писать в телеграм.*

**Далее идут блоки, разбитые по разделам. Нулевой раздел выполняется обязательно. Остальные по желанию.**

# <b><font color="gree" size="+3">0. Подготовка к обработке</b>

**ОБЯЗАТЕЛЬНЫЙ РАЗДЕЛ !**

*↓ Нажмите, чтобы открыть раздел ↓*

In [ ]:
#@title ##**Устанавливаем все необходимые библиотеки** { display-mode: "form" }
!pip install youtube_dl
!pip install ffmpeg
!pip install ffmpeg-python
!pip install torchvision==0.5
!pip install torch==1.4

from IPython.display import clear_output
from google.colab import files
import imageio
import youtube_dl
import cv2
import os
import torch
import fastai
import ffmpeg
import os.path as osp
import logging
import shutil
import re
import gc
from PIL import Image
from tqdm import tqdm
from os import path
import numpy as np
import moviepy.editor as mpy
from moviepy.video.io.ffmpeg_writer import FFMPEG_VideoWriter
from pathlib import Path
import sys
import glob
from IPython import display as ipythondisplay
from IPython.display import Image as ipythonimage
torch.backends.cudnn.benchmark=True
%matplotlib inline

!rm -rf sample_data
clear_output()

In [ ]:
#@title ##**Загружаем видео** { display-mode: "form" }
#@markdown *Ниже введите ссылку на видео (например, YouTube или Twitter), либо оставьте поле **source_url** пустым (в таком случае будет предложено загрузить ролик с компьютера).*
source_url = '' #@param {type:"string"}

if source_url == '':
  uploaded = files.upload()
  for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
        name=fn, length=len(uploaded[fn])))
  os.rename(fn, fn.replace(" ", ""))
  fn = fn.replace(" ", "")
  file_name = "downloaded_video." + fn.split(".")[-1]
  !mv -f $fn $file_name

else:
  try:
    ydl_opts = {
        'format': 'bestvideo[ext=mp4]+bestaudio[ext=m4a]/mp4',
        'outtmpl': 'downloaded_video.mp4',
        }
    with youtube_dl.YoutubeDL(ydl_opts) as ydl:
      ydl.download([source_url])
    file_name = 'downloaded_video.mp4'
  
  except BaseException:
    !wget $source_url
    fn = source_url.split('/')[-1]
    os.rename(fn, fn.replace(" ", ""))
    fn = fn.replace(" ", "")
    file_name = "downloaded_video." + fn.split(".")[-1]
    !mv -f $fn $file_name

!cp -r downloaded_video.mp4 video.mp4
clear_output()
fps_of_video = int(cv2.VideoCapture(file_name).get(cv2.CAP_PROP_FPS))
frames_of_video = int(cv2.VideoCapture(file_name).get(cv2.CAP_PROP_FRAME_COUNT))
#@markdown *Не рекомендуется загружать видео, длящиеся дольше одной минуты и содержащие в названии файла точки или пробелы.*

#@markdown *Если при выполнении возникнет ошибка, то запустите этот блок заново*

In [ ]:
#@title ##**Обрезка видео (Необязательный блок)** { display-mode: "form" }
#@markdown *Здесь можно обрезать видео если оно слишком длинное.*

#@markdown **Введите временной отрезок для обрезки видео**
target_start = '00:00:00' #@param {type:"string"}
target_end = '00:00:05' #@param {type:"string"}

#@markdown Введите новое разрешение видео файла для рескейлинга (*например 640 и 480*), или оставьте пустым чтобы работать с оригинальным разрешением
width = '' #@param {type:"string"}
height = '' #@param {type:"string"}

rescale = ""
if width != '' and height != '':
  rescale = f"-s {width}x{height}"

!ffmpeg -i downloaded_video.mp4 $rescale -ss $target_start -to $target_end new_target.mp4
clear_output()
!rm video.mp4
!mv new_target.mp4 video.mp4
#@markdown *Отобразить видео:*
show_video = False #@param {type:"boolean"}
if show_video == True:
  display(mpy.ipython_display("video.mp4", height=400))

**Следующие разделы вы можете выполнять в любом порядке (главное каждый блок внутри раздела выполнять поочередно).**

---
# <b><font color="gree" size="+3">1. Раскрашивание видео ([DeOldify](https://github.com/jantic/DeOldify))
*↓ Нажмите, чтобы открыть раздел ↓*


Автор: [Jason Antic](https://twitter.com/citnaj)

In [ ]:
#@title ##**Клонируем репозиторий и устанавливаем необходимые зависимости** { display-mode: "form" }
%cd /content
!git clone https://github.com/jantic/DeOldify.git
%cd /content/DeOldify
!pip install -r colab_requirements.txt
clear_output()

In [ ]:
#@title ##**Качаем предобученную модель** { display-mode: "form" }
from deoldify.visualize import *
!mkdir 'models'
try:
  !rm -rf models/ColorizeVideo_gen.pth
  !gdown https://drive.google.com/uc?id=1-3ONnPYcX9fOqnY-pUKYGJOd6XeFon9X -O ./models/ColorizeVideo_gen.pth
except BaseException:
  !rm -rf models/ColorizeVideo_gen.pth
  !wget https://www.dropbox.com/s/336vn9y4qwyg9yz/ColorizeVideo_gen.pth?dl=0 -O ./models/ColorizeVideo_gen.pth
colorizer = get_video_colorizer()
clear_output()

In [ ]:
#@title ##**Раскрашиваем видео** { display-mode: "form" }
#@markdown **21, 35** *- оптимальные показатели*
!rm -rf video
!mkdir 'video'
!mkdir 'video/source'
!cp -r /content/video.mp4 video/source/video.mp4

render_factor = 21  #@param {type: "slider", min: 5, max: 44}
video_path = colorizer.colorize_from_file_name('video.mp4', render_factor)
clear_output()

Если при выполнении блока сверху вылетает ошибка, то конвертируйте видео [ТУТ](https://convert-video-online.com/ru/) и начните выполнение всего скрипта заново.

In [ ]:
#@title ##**Получаем результат** { display-mode: "form" }
#@markdown **play** *- воспроизвести видео в браузере,*

#@markdown **download** *- загрузить на компьютер*
!cp -r video/result/video.mp4 /content/colorized_video.mp4
what_next = 'play' #@param ["play", "download"]
if what_next == "play":
  display(mpy.ipython_display("/content/colorized_video.mp4", height=400, autoplay=1, loop=1, maxduration=600))
else:
  files.download('/content/colorized_video.mp4')

In [ ]:
#@title ###**Не обязательный блок!** { display-mode: "form" }
#@markdown *Если вам кажется, что качество раскрашивания недостаточное, то запустите этот блок, дождитесь полного его завершения и посмотрите, при каком значении* **render_factor** *картинка вам нравится больше. Далее запустите предыдущий блок раскрашивание заново, установив нужное значение.*
for i in range(10,45,2):
    colorizer.vis.plot_transformed_image('video/bwframes/video/00001.jpg', render_factor=i, display_render_factor=True, figsize=(8,8))

In [ ]:
#@title ##**Выход из раздела Раскрашивания видео** { display-mode: "form" }
#@markdown *Если вы собираетесь продолжить выполнять следующие пункты, то прежде всего обязательно выполните этот блок!*
!rm -rf /content/video.mp4
!cp -r video/result/video.mp4 /content/video.mp4
!cp -r video/result/video.mp4 /content/colorized_video.mp4
%cd /content
clear_output()


---
# <b><font color="gree" size="+3">2. Увеличение детализации видео ([EDVR](https://github.com/xinntao/EDVR))
*↓ Нажмите, чтобы открыть раздел ↓*

Автор: [Xintao Wang](https://xinntao.github.io)

In [ ]:
#@title ##**Клонируем репозиторий и качаем предобученную модель** { display-mode: "form" }
%cd /content
!pip install numpy opencv-python lmdb pyyaml
!pip install tb-nightly future
!git clone https://github.com/xinntao/EDVR
%cd /content/EDVR/codes/models/archs/dcn
!python setup.py develop
%cd /content/EDVR/codes
!wget --no-check-certificate 'https://drive.google.com/uc?export=download&id=1SGVehpZt4WL_X8Jh6blyqmHpc8DdImgv' -O /content/EDVR/experiments/pretrained_models/EDVR_REDS_deblurcomp_L.pth
clear_output()

In [ ]:
#@title ##**Инициализируем необходимые функции** { display-mode: "form" }
import utils.util as util
import data.util as data_util
import models.archs.EDVR_arch as EDVR_arch

workfolder = Path('./video')
source_folder = workfolder / "source"
inframes_root = workfolder / "inframes"
audio_root = workfolder / "audio"
outframes_root = workfolder / "outframes"
result_folder = workfolder / "result"
pretrained_models = Path('../experiments/pretrained_models')

def clean_mem():
    # torch.cuda.empty_cache()
    gc.collect()

def get_fps(source_path: Path) -> str:
    print(source_path)
    probe = ffmpeg.probe(str(source_path))
    stream_data = next(
        (stream for stream in probe['streams'] if stream['codec_type'] == 'video'),
        None,
    )
    return stream_data['avg_frame_rate']

def preProcess(imag_path_l, multiple):
  '''Need to resize images for blurred model (needs to be multiples of 16)'''
  for img_path in imag_path_l:
    im = Image.open(img_path)
    h, w = im.size
    # resize so they are multiples of 4 or 16 (for blurred)
    h = h - h % multiple
    w = w - w % multiple
    im = im.resize((h,w))
    im.save(img_path)

def purge_images(dir):
  for f in os.listdir(dir):
    if re.search('.*?\.jpg', f):
      os.remove(os.path.join(dir, f))

def extract_raw_frames(source_path: Path):
    inframes_folder = inframes_root / (source_path.stem)
    inframe_path_template = str(inframes_folder / '%5d.jpg')
    inframes_folder.mkdir(parents=True, exist_ok=True)
    purge_images(inframes_folder)
    ffmpeg.input(str(source_path)).output(
        str(inframe_path_template), format='image2', vcodec='mjpeg', qscale=0
    ).run(capture_stdout=True)

def make_subfolders(img_path_l, chunk_size):
  i = 0
  subFolderList = []
  source_img_path = Path('/content/EDVR/codes/video/inframes/video_subfolders')
  source_img_path.mkdir(parents=True, exist_ok=True)
  for img in img_path_l:
    if i % chunk_size == 0:
      img_path = source_img_path / str(i)
      img_path.mkdir(parents=True, exist_ok=True)
      subFolderList.append(str(img_path))
    i+=1
    img_name = osp.basename(img)
    img_path_name = img_path / img_name
    shutil.copyfile(img, img_path_name)

  return subFolderList

def remove_subfolders():
  shutil.rmtree('/content/EDVR/codes/video/inframes/video_subfolders', ignore_errors=True, onerror=None)

def edvrPredict(data_mode, chunk_size, stage):
  device = torch.device('cuda')
  os.environ['CUDA_VISIBLE_DEVICES'] = '0'
  data_mode = "blur_comp"
  stage = stage  # 1 or 2, use two stage strategy for REDS dataset.
  flip_test = False
  model_path = pretrained_models / 'EDVR_REDS_deblurcomp_L.pth'   

  print('Model Used: ', model_path)
  
  if data_mode == 'Vid4':
      N_in = 7  # use N_in images to restore one HR image
  else:
      N_in = 5

  predeblur, HR_in = False, False
  back_RBs = 40
  if data_mode == 'blur' or data_mode == 'blur_comp':
      predeblur, HR_in = True, True
  if stage == 2:
      HR_in = True
      back_RBs = 20
  if data_mode == 'TOF':
    model = TOF_arch.TOFlow(adapt_official=True)
  else:
    model = EDVR_arch.EDVR(128, N_in, 8, 5, back_RBs, predeblur=predeblur, HR_in=HR_in)

  #### dataset
  test_dataset_folder = '/content/EDVR/codes/video/inframes'

  #### evaluation
  crop_border = 0
  border_frame = N_in // 2  # border frames when evaluate
  # temporal padding mode
  if data_mode in ('Vid4','sharp_bicubic'):
      padding = 'new_info'
  else:
      padding = 'replicate'
  save_imgs = True

  save_folder = '/content/EDVR/codes/video/outframes'
  util.mkdirs(save_folder)

  #### set up the models
  model.load_state_dict(torch.load(model_path), strict=True)
  model.eval()
  model = model.to(device)

  avg_psnr_l, avg_psnr_center_l, avg_psnr_border_l = [], [], []
  subfolder_name_l = []
  # remove old video_subfolder if exists
  remove_subfolders()
  subfolder_l = sorted(glob.glob(osp.join(test_dataset_folder, '*')))

  # for each subfolder
  for subfolder in subfolder_l:
      subfolder_name = osp.basename(subfolder)
      subfolder_name_l.append(subfolder_name)
      save_subfolder = osp.join(save_folder, subfolder_name)

      img_path_l = sorted(glob.glob(osp.join(subfolder, '*')))
      if save_imgs:
          util.mkdirs(save_subfolder)
          purge_images(save_subfolder)

      # preprocess images (needed for blurred models)
      if predeblur:
        preProcess(img_path_l, 16)
      else:
        preProcess(img_path_l, 4)
      # make even more subfolders
      subFolderList = make_subfolders(img_path_l, chunk_size)

      #### read LQ and GT images in chunks of 1000
      for subSubFolder in subFolderList:
        clean_mem()
        imgs_LQ = data_util.read_img_seq(subSubFolder)
        subSubFolder_l = sorted(glob.glob(osp.join(subSubFolder, '*')))
        max_idx = len(subSubFolder_l)
        avg_psnr, avg_psnr_border, avg_psnr_center, N_border, N_center = 0, 0, 0, 0, 0

        # process each image
        for img_idx, img_path in tqdm(enumerate(subSubFolder_l)):
            img_name = osp.splitext(osp.basename(img_path))[0]
            select_idx = data_util.index_generation(img_idx, max_idx, N_in, padding=padding)
            imgs_in = imgs_LQ.index_select(0, torch.LongTensor(select_idx)).unsqueeze(0).to(device)

            if flip_test:
                output = util.flipx4_forward(model, imgs_in)
            else:
                output = util.single_forward(model, imgs_in)
            output = util.tensor2img(output.squeeze(0))

            if save_imgs:
                cv2.imwrite(osp.join(save_subfolder, '{}.jpg'.format(img_name)), output)
                # print('Saved Image:', str(osp.join(save_subfolder, '{}.jpg'.format(img_name))))

def moveProcessedFrames():
  shutil.rmtree('/content/EDVR/codes/video/inframes')
  os.rename('/content/EDVR/codes/video/outframes', '/content/EDVR/codes/video/inframes')

def build_video(source_path: Path) -> Path:
        out_path = result_folder / (
            source_path.name.replace('.mp4', '_no_audio.mp4')
        )
        outframes_folder = outframes_root / (source_path.stem)
        outframes_path_template = str(outframes_folder / '%5d.jpg')
        out_path.parent.mkdir(parents=True, exist_ok=True)
        if out_path.exists():
            out_path.unlink()
        fps = get_fps(source_path)
        print('Original FPS is: ', fps)

        ffmpeg.input(
            str(outframes_path_template),
            format='image2',
            vcodec='mjpeg',
            framerate=fps,
        ).output(str(out_path), crf=17, vcodec='libx264').run(capture_stdout=True)

        result_path = result_folder / source_path.name
        if result_path.exists():
            result_path.unlink()
        # making copy of non-audio version in case adding back audio doesn't apply or fails.
        shutil.copyfile(str(out_path), str(result_path))

        # adding back sound here
        audio_file = Path(str(source_path).replace('.mp4', '.aac'))
        if audio_file.exists():
            audio_file.unlink()

        os.system(
            'ffmpeg -y -i "'
            + str(source_path)
            + '" -vn -acodec copy "'
            + str(audio_file)
            + '"'
        )

        if audio_file.exists:
            os.system(
                'ffmpeg -y -i "'
                + str(out_path)
                + '" -i "'
                + str(audio_file)
                + '" -shortest -c:v copy -c:a aac -b:a 256k "'
                + str(result_path)
                + '"'
            )
        print('Video created here: ' + str(result_path))
        return result_path

def edvr_video(source_path: Path, chunk_size: int):
    # extract frames
    extract_raw_frames(source_path)

    # process frames
    edvrPredict("blur_comp", chunk_size, 1)

    # build back video
    build_video(source_path)


In [ ]:
#@title ##**Улучшаем качество кадров** { display-mode: "form" }
%%time
edvr_video(Path('/content/video.mp4'), 100)
clear_output()

In [ ]:
#@title ##**Получаем результат** { display-mode: "form" }
#@markdown **play** *- воспроизвести видео в браузере,*

#@markdown **download** *- загрузить на компьютер*
!rm -rf /content/video.mp4
!cp -r /content/EDVR/codes/video/result/video.mp4 /content/video.mp4
!cp -r /content/EDVR/codes/video/result/video.mp4 /content/enhanced_video.mp4
what_next = 'play' #@param ["play", "download"]
if what_next == "play":
  display(mpy.ipython_display("/content/enhanced_video.mp4", height=400, autoplay=1, loop=1, maxduration=600))
else:
  files.download('/content/enhanced_video.mp4')

In [ ]:
#@title ##**Выход из раздела Увеличения детализации видео** { display-mode: "form" }
#@markdown *Если вы собираетесь продолжить выполнять следующие пункты, то прежде всего обязательно выполните этот блок!*
!rm -rf /content/video.aac
!cp -r /content/EDVR/codes/video/result/video.mp4 /content/video.mp4
!cp -r /content/EDVR/codes/video/result/video.mp4 /content/enhanced_video.mp4
%cd /content
clear_output()


---
# <b><font color="gree" size="+3">3. Интерполяция кадров ([VFIASC](https://github.com/sniklaus/sepconv-slomo))

*(Ниже в пункте "Дополнительные разделы" можно найти другой метод интерполяции. Он выполняется быстрее, но менее качественно)*

*↓ Нажмите, чтобы открыть раздел ↓*

Автор: [Simon Niklaus](http://sniklaus.com/about/welcome)

In [ ]:
#@title ##**Клонируем репозиторий и качаем необходимые компоненты** { display-mode: "form" }
%cd /content
!git clone https://github.com/sniklaus/sepconv-slomo.git VFIASC
%cd /content/VFIASC
!rm -rf network-l1.pytorch
!rm -rf network-lf.pytorch
try:
  !gdown https://drive.google.com/uc?id=1v77wNU8sYh0hBmljeGt5LzY9ehQU0elv
  !gdown https://drive.google.com/uc?id=1na11Ey0TB1KEDps4uEwyTBsDcmpo2kr6
except BaseException:
  !bash download.bash
clear_output()

In [ ]:
#@title ##**Устанавливаем необходимые зависимости** { display-mode: "form" }
#@markdown **Данный блок обязательный. Время его выполнения: ~12 минут**
print('Можете пока сходить за кофейком.\n')
!pip install cupy
!pip install moviepy
clear_output()
print('Ну вот и все! Пора выполнять следующий блок.')

In [ ]:
#@title ##**Интерполируем кадры** { display-mode: "form" }
#@markdown **Во сколько раз увеличить FPS:**
fps_boosting = 'x4' #@param ["x4", "x16"]
%env CUDA_VISIBLE_DEVICES=0
if fps_boosting == 'x4':
  !python run.py --model lf --video /content/video.mp4 --out ./video_x4.mp4
  video_name = "video_x4.mp4"
elif fps_boosting == 'x16':
  !python run.py --model lf --video /content/video.mp4 --out ./video_x4.mp4
  !python run.py --model lf --video ./video_x4.mp4 --out ./video_x16.mp4
  video_name = "video_x16.mp4"
#clear_output()

mfps_of_video = int(cv2.VideoCapture(video_name).get(cv2.CAP_PROP_FPS))
mframes_of_video = int(cv2.VideoCapture(video_name).get(cv2.CAP_PROP_FRAME_COUNT))
#@markdown *Рекомендуется устанавливать* **x4.** *В противном случае выполнение этого блока может затянуться или привести к ошибке. Ко всему прочему эффект СлоуМо будет неестественно сильным*

In [ ]:
#@title ##**Получаем результат** { display-mode: "form" }
#@markdown **play** *- воспроизвести видео в браузере,*

#@markdown **download** *- загрузить на компьютер*
!rm -rf /content/video.mp4
!cp -r $video_name /content/video.mp4
!cp -r $video_name /content/fps_boosted_video.mp4
what_next = 'play' #@param ["play", "download"]
if what_next == "play":
  display(mpy.ipython_display("/content/fps_boosted_video.mp4", height=400, autoplay=1, loop=1, maxduration=600))
else:
  files.download("/content/fps_boosted_video.mp4")

In [ ]:
#@title ##**Конвертируем СлоуМо видео в высокочастотное и автоматически предлагаем скачать его на компьютер** { display-mode: "form" }
!rm -rf output.mp4
!rm -rf slowed_movie_frames
!rm -rf /content/video.mp4
!rm -rf /content/fps_boosted_video.mp4

fr = int(fps_of_video*int(fps_boosting.split("x")[1]))
!mkdir 'slowed_movie_frames'
vidcap = cv2.VideoCapture(video_name)
success,image = vidcap.read()
count = 0
success = True
while success:
  cv2.imwrite("slowed_movie_frames/frame%09d.jpg" % count, image)
  success,image = vidcap.read()
  count += 1

staffs = []
img = os.listdir("slowed_movie_frames/")
img.sort()
clear_output()
for i in img:
  staffs.append("slowed_movie_frames/"+i)

staff = cv2.imread(staffs[0])  # get size from the 1st frame
writer = cv2.VideoWriter(
    'output.mp4',
    cv2.VideoWriter_fourcc(*'MP4V'),   # codec (*'DIVX', *'MP4V', *'FMP4', *'MJPG', *'XVID', *'MP4S')
    fr,  # fps
    (staff.shape[1], staff.shape[0]),  # width, height
    isColor=len(staff.shape) > 2)
for staff in map(cv2.imread, staffs):
    writer.write(staff)
writer.release()
clear_output()

pfps_of_video = int(cv2.VideoCapture("output.mp4").get(cv2.CAP_PROP_FPS))
pframes_of_video = int(cv2.VideoCapture("output.mp4").get(cv2.CAP_PROP_FRAME_COUNT))

!cp -r output.mp4 /content/fps_boosted_video.mp4
!cp -r output.mp4 /content/video.mp4
files.download("/content/fps_boosted_video.mp4")

In [ ]:
#@title ##**Выход из раздела Интерполяции кадров** { display-mode: "form" }
#@markdown *Если вы собираетесь продолжить выполнять следующие пункты, то прежде всего обязательно выполните этот блок!*
%cd /content/
clear_output()


---
# <b><font color="gree" size="+3">4. Увеличение разрешения ([ESRGAN](https://github.com/xinntao/ESRGAN))

*↓ Нажмите, чтобы открыть раздел ↓*

Автор: [Xintao Wang](https://xinntao.github.io)

In [ ]:
#@title ##**Клонируем репозиторий и качаем необходимые компоненты** { display-mode: "form" }
%cd /content
!git clone https://github.com/xinntao/ESRGAN.git
!cp -r video.mp4 /content/ESRGAN/
%cd /content/ESRGAN
!git checkout tags/old-arch
model_url = "https://www.dropbox.com/s/vouc15j8jjp2o5n/RRDB_ESRGAN_x4_old_arch.pth?dl=0"
!wget $model_url --content-disposition -P models
import architecture as arch
import os.path
!mkdir frames
!rm -rf results/baboon_ESRGAN.png
clear_output()

In [ ]:
#@title ##**Разделяем видео на кадры** { display-mode: "form" }
frames_of_video = int(cv2.VideoCapture("video.mp4").get(cv2.CAP_PROP_FRAME_COUNT))
fps_of_video = int(cv2.VideoCapture("video.mp4").get(cv2.CAP_PROP_FPS))
vidcap = cv2.VideoCapture('video.mp4')
success,image = vidcap.read()
count = 0
success = True
while success:
  cv2.imwrite("frames/frame%09d.jpg" % count, image)
  success,image = vidcap.read()
  count += 1

In [ ]:
#@title ##**Увеличиваем разрешение кадра** { display-mode: "form" }
#@markdown **Во сколько раз увеличить разрешение:**
upscale = 4 #@param {type: "slider", min: 4, max: 8}
%env CUDA_VISIBLE_DEVICES=0
device = torch.device('cuda')
model = arch.RRDB_Net(3, 3, 64, 23, gc=32, upscale=upscale, norm_type=None, act_type='leakyrelu', \
                        mode='CNA', res_scale=1, upsample_mode='upconv')
model.load_state_dict(torch.load('models/{:s}'.format('RRDB_ESRGAN_x4_old_arch.pth')), strict=True)
model.eval()
for k, v in model.named_parameters():
    v.requires_grad = False
model = model.to(device)

count_frames = 0

for path in glob.glob('frames/*'):
    base = os.path.splitext(os.path.basename(path))[0]
    img = cv2.imread(path, cv2.IMREAD_COLOR)
    img = img * 1.0 / 255
    img = torch.from_numpy(np.transpose(img[:, :, [2, 1, 0]], (2, 0, 1))).float()
    img_LR = img.unsqueeze(0)
    img_LR = img_LR.to(device)

    output = model(img_LR).data.squeeze().float().cpu().clamp_(0, 1).numpy()
    output = np.transpose(output[[2, 1, 0], :, :], (1, 2, 0))
    output = (output * 255.0).round()
    path = 'results/{:s}_rlt.png'.format(base)
    cv2.imwrite(path, output)
    count_frames += 1
    clear_output()
    print("Обработано: {} из {}".format(str(count_frames), str(frames_of_video)))
clear_output()

In [ ]:
#@title ##**Собираем кадры в видео** { display-mode: "form" }
frames = []
img = os.listdir("results/")
img.sort()
for i in img:
  frames.append(imageio.imread("results/"+i))
frames = np.array(frames)
imageio.mimsave("upscaled_video.mp4", frames, fps=fps_of_video)

print('Сборка завершена')
!cp -r upscaled_video.mp4 /content/upscaled_videod.mp4
!cp -r upscaled_video.mp4 /content/video.mp4
clear_output()

In [ ]:
#@title ##**Получаем результат** { display-mode: "form" }
#@markdown **play** *- воспроизвести видео в браузере,*

#@markdown **download** *- загрузить на компьютер*
what_next = 'download' #@param ["play", "download"]
if what_next == "play":
  display(mpy.ipython_display("/content/upscaled_video.mp4", height=400, autoplay=1, loop=1, maxduration=600))
else:
  files.download("/content/upscaled_video.mp4")

In [ ]:
#@title ##**Выход из раздела Высококачественного разрешения** { display-mode: "form" }
#@markdown *Если вы собираетесь продолжить выполнять следующие пункты, то прежде всего обязательно выполните этот блок!*
%cd /content
clear_output()


# <b><font color="black" size="+3">Дополнительные разделы
<font color="grey">В этих разделах находятся нейросети, которые не вошли в основной перечень инструментов обработки видео по разным причинам.

*↓ Нажмите, чтобы открыть дополнительные разделы ↓*

---
## <b><font color="grey" size="+2">Удаление мелких помех ([DeepRemaster](https://github.com/satoshiiizuka/siggraphasia2019_remastering))
*↓ Нажмите, чтобы открыть раздел ↓*

Автор: [Satoshi Iizuka](http://iizuka.cs.tsukuba.ac.jp/index_eng.html)

In [ ]:
#@title ##**Клонируем репозиторий** { display-mode: "form" }
%cd /content
!git clone https://github.com/satoshiiizuka/siggraphasia2019_remastering.git DeepRemaster
!cp -r /content/video.mp4 /content/DeepRemaster/
%cd /content/DeepRemaster
clear_output()

In [ ]:
#@title ##**Качаем предобученную модель** { display-mode: "form" }
try:
  !rm -rf model/remasternet.pth.tar
  !gdown https://drive.google.com/uc?id=1pB8C7c2EDUXNL0Ytk0NZ8q8bAj_nFykO
  !gdown https://drive.google.com/uc?id=1fJJK6i6jVGJmWhoHssuHMZFmfnULxe8S -O model/remasternet.pth.tar
  !bash ./checking_model.sh
except BaseException:
  !rm -rf model/remasternet.pth.tar
  !bash ./download_model.sh
clear_output()

In [ ]:
#@title ##**Удаляем помехи на кадрах** { display-mode: "form" }
!python remaster.py --input video.mp4 --disable_colorization --gpu
clear_output()

In [ ]:
#@title ##**Получаем результат** { display-mode: "form" }
#@markdown **play** *- воспроизвести видео в браузере,*

#@markdown **download** *- загрузить на компьютер*
!rm -rf /content/video.mp4
!cp -r video_out.mp4 /content/video.mp4
!cp -r video_out.mp4 /content/denoise_video.mp4
what_next = 'play' #@param ["play", "download"]
if what_next == "play":
  display(mpy.ipython_display("/content/denoise_video.mp4", height=400, autoplay=1, loop=1, maxduration=600))
else:
  files.download('/content/denoise_video.mp4')

In [ ]:
#@title ##**Выход из раздела Удаления мелких помех** { display-mode: "form" }
#@markdown *Если вы собираетесь продолжить выполнять следующие пункты, то прежде всего обязательно выполните этот блок!*
!cp -r video_out.mp4 /content/video.mp4
!cp -r video_out.mp4 /content/denoise_video.mp4
%cd /content
clear_output()


---
## <b><font color="grey" size="+2">Увеличение резрешения кадров ([Fast-SRGAN](https://github.com/HasnainRaz/Fast-SRGAN))
*Если надумаете выполнять этот блок, то рекомендуется делать это послу предыдущего раздела "Удаление помех"*

*↓ Нажмите, чтобы открыть раздел ↓*

Автор: [Hasnain Raza](https://github.com/HasnainRaz)

In [ ]:
#@title ###**Клонируем репозиторий и устанавливаем необходимые зависимости** { display-mode: "form" }
%cd /content
!git clone https://github.com/HasnainRaz/Fast-SRGAN.git
!cp -r /content/video.mp4 /content/Fast-SRGAN/
%cd /content/Fast-SRGAN
!pip install -r requirements.txt
clear_output()

In [ ]:
#@title ###**Генерируем кадры высокого разрешения** { display-mode: "form" }
#@markdown *Данный блок работает весьма медленно,а также качество итоговых изображений весьма спорное, поэтому я и вынес этот раздел в Дополнительные.*
!rm -rf hr_frames
!rm -rf frames
!mkdir 'hr_frames'
!mkdir 'frames'

vidcap = cv2.VideoCapture('video.mp4')
success,image = vidcap.read()
count = 0
success = True
while success:
  cv2.imwrite("frames/frame%09d.jpg" % count, image)
  success,image = vidcap.read()
  count += 1

!python infer.py --image_dir 'frames/' --output_dir 'hr_frames/'
clear_output()

In [ ]:
#@title ###**Собираем кадры в единое видео** { display-mode: "form" }

!rm -rf HR_video.mp4
!rm -rf ../HR_video.mp4
frames = []
img = os.listdir("hr_frames/")
img.sort()
for i in img:
  frames.append(imageio.imread("hr_frames/"+i))
frames = np.array(frames)
imageio.mimsave("HR_video.mp4", frames)

print('Сборка завершена')
!cp -r HR_video.mp4 ../HR_video.mp4
clear_output()

In [ ]:
#@title ###**Получаем результат** { display-mode: "form" }
#@markdown **play** *- воспроизвести видео в браузере,*

#@markdown **download** *- загрузить на компьютер*
what_next = 'play' #@param ["play", "download"]
if what_next == "play":
  display(mpy.ipython_display("HR_video.mp4", height=400, autoplay=1, loop=1, maxduration=600))
else:
  files.download("HR_video.mp4")

In [ ]:
#@title ###**Конвертируем СлоуМо видео в высокочастотное и автоматически предлагаем скачать его на компьютер** { display-mode: "form" }
fps_of_video = int(cv2.VideoCapture("video.mp4").get(cv2.CAP_PROP_FPS))

!rm -rf HR_video.mp4
!rm -rf ../HR_video.mp4
frames = []
img = os.listdir("hr_frames/")
img.sort()
clear_output()
for i in img:
  frames.append("hr_frames/"+i)
frames = np.array(frames)

frame = cv2.imread(frames[0])  # get size from the 1st frame
writer = cv2.VideoWriter(
    'HR_video.mp4',
    cv2.VideoWriter_fourcc(*'MP4V'),   # codec (*'DIVX', *'MP4V', *'FMP4', *'MJPG', *'XVID', *'MP4S')
    fps_of_video,  # fps
    (frame.shape[1], frame.shape[0]),  # width, height
    isColor=len(frame.shape) > 2)
for frame in map(cv2.imread, frames):
    writer.write(frame)
writer.release()

clear_output()
print('Конвертирование завершено. Ожидайте начала загрузки (при появлении можно нажать Отмена).')
!cp -r HR_video.mp4 /content/HR_video.mp4
files.download("HR_video.mp4")
clear_output()

In [ ]:
#@title ###**Выход из раздела Увеличения разрешения** { display-mode: "form" }
#@markdown *Если вы собираетесь продолжить выполнять следующие пункты, то прежде всего обязательно выполните этот блок!*
!rm -rf /content/video.mp4
!cp -r HR_video.mp4 /content/video.mp4
%cd /content/
clear_output()


---
## <b><font color="grey" size="+2">Быстрая, но менее качественная интерполяция кадров ([VFI-CFT](https://github.com/MortenHannemose/pytorch-vfi-cft))
*↓ Нажмите, чтобы открыть раздел ↓*

Автор: [Morten Hannemose](https://scholar.google.ru/citations?user=AH58CjUAAAAJ&hl=ru)

In [ ]:
#@title ###**Клонируем репозиторий и качаем предобученную модель** { display-mode: "form" }
%cd /content
!git clone https://github.com/MortenHannemose/pytorch-vfi-cft.git VFI-CFT
%cd /content/VFI-CFT
video = "video_2x_slow.mp4"
try:
  !rm -rf VFI_CFT_weights.pt.gz
  !rm -rf slow_movie.py
  !gdown https://drive.google.com/uc?id=1-5zS1XOhwVXJhD4JDZWmG6plC_IVVg8b
  !gdown https://drive.google.com/uc?id=1YinJeeUbTbDynD6kw6f6h76CAkRqFdHK
except BaseException:
  !rm -rf VFI_CFT_weights.pt.gz
  !gdown https://drive.google.com/uc?id=1rQAi0UXcaMcEo8_l4oSd8vJUVzp2dIIF
clear_output()

In [ ]:
#@title ###**Интерполируем кадры** { display-mode: "form" }
#@markdown **Во сколько раз увеличить FPS:**
!rm -rf slowed_movie_frames
!rm -rf $video
!rm -rf video.mp4
!cp -r /content/video.mp4 /content/VFI-CFT
fps_boosting = 2 #@param {type:"slider", min:2, max:8, step:1}

!python slow_movie.py -m video.mp4 -f $fps_boosting

video = "video_"+str(fps_boosting)+"x_slow.mp4"
frames = []
img = os.listdir("slowed_movie_frames/")
img.sort()
clear_output()
for i in img:
  frames.append(imageio.imread("slowed_movie_frames/"+i))
frames = np.array(frames)
imageio.mimsave("fps_boosted_video.mp4", frames)
#@markdown *Более 4 ставить не рекомендуется. Эффект СлоуМо будет неестественно сильным*
clear_output()

nfps_of_video = int(cv2.VideoCapture("fps_boosted_video.mp4").get(cv2.CAP_PROP_FPS))
nframes_of_video = int(cv2.VideoCapture("fps_boosted_video.mp4").get(cv2.CAP_PROP_FRAME_COUNT))

In [ ]:
#@title ###**Получаем результат** { display-mode: "form" }
#@markdown **play** *- воспроизвести видео в браузере,*

#@markdown **download** *- загрузить на компьютер*
!rm -rf /content/video.mp4
!cp -r fps_boosted_video.mp4 /content/video.mp4
!cp -r fps_boosted_video.mp4 /content/fps_boosted_video.mp4
what_next = 'play' #@param ["play", "download"]
if what_next == "play":
  display(mpy.ipython_display("/content/fps_boosted_video.mp4", height=400, autoplay=1, loop=1, maxduration=600))
else:
  files.download("/content/fps_boosted_video.mp4")

In [ ]:
#@title ###**Выход из раздела Интерполяции кадров** { display-mode: "form" }
#@markdown *Если вы собираетесь продолжить выполнять следующие пункты, то прежде всего обязательно выполните этот блок!*
%cd /content/
clear_output()


---
## <b><font color="grey" size="+2">Бонус. Расскрашивание фото и Параллакс эффект  ([DeOldify](https://github.com/jantic/DeOldify) + [3D Ken Burns](https://github.com/sniklaus/3d-ken-burns))</b>
Данный раздел можно выполнять самым первым. По его окончанию, полученный видеоролик можно отраставрировать в других разделах. Важно, перед прохождением по другим разделам необходимо зайти в **"0. Подготовка к обработке"** и выполнить блок **"Устанавливаем все необходимые библиотеки"**. Далее все остальные разделы будут нормально функционировать.

*↓ **Нажмите, чтобы открыть раздел** ↓*

In [2]:
#@title ###**Загружаем фото** { display-mode: "form" }
#@markdown *Ниже введите ссылку на фото, либо оставьте поле **source_url** пустым (в таком случае будет предложено загрузить ролик с компьютера).*
from IPython.display import clear_output
from google.colab import files
import os
%cd /content
!rm -rf sample_data
source_url = 'http://www.worldfocus.ru/_ph/1/444433832.jpg' #@param {type:"string"}

if source_url == '':
  uploaded = files.upload()
  for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
        name=fn, length=len(uploaded[fn])))
  os.rename(fn, fn.replace(" ", ""))
  fn = fn.replace(" ", "")
  pic_name = "photo." + fn.split(".")[-1]
  !mv -f $fn $pic_name

else:
  try:
    pic_name = "photo." + source_url.split('.')[-1]
    !wget $source_url -O $pic_name
  except BaseException:
    print('Что-то пошло не так. Попробуйте загрузить фото с компьютера.')

!cp -r photo.jpg downloaded_photo.jpg
clear_output()
#@markdown *Если при выполнении возникнет ошибка, то запустите этот блок заново*

### <b><font color="aquamarine" size="+1">1) DeOldify - Раскрашивание фото</b>
Автор DeOldify: [Jason Antic](https://twitter.com/citnaj)

*↓ Нажмите, чтобы открыть подраздел ↓*

In [ ]:
#@title ###**Клонируем репозиторий и устанавливаем необходимые зависимости** { display-mode: "form" }
import torch
import fastai
%cd /content
!git clone https://github.com/jantic/DeOldify.git DeOldifyPhoto
%cd /content/DeOldifyPhoto
from deoldify import device
from deoldify.device_id import DeviceId
device.set(device=DeviceId.GPU0)
if not torch.cuda.is_available():
    print('GPU not available.')
!pip install -r colab_requirements.txt
clear_output()

In [ ]:
#@title ###**Качаем предобученную модель** { display-mode: "form" }
from deoldify.visualize import *
torch.backends.cudnn.benchmark = True
!mkdir 'models'
!rm -rf models/ColorizeStable_gen.pth
!wget https://www.dropbox.com/s/mwjep3vyqk5mkjc/ColorizeStable_gen.pth?dl=0 -O ./models/ColorizeStable_gen.pth
!wget https://media.githubusercontent.com/media/jantic/DeOldify/master/resource_images/watermark.png -O ./resource_images/watermark.png
colorizer = get_image_colorizer(artistic=False)
clear_output()

In [ ]:
#@title ###**Раскрашиваем фото** { display-mode: "form" }
#@markdown **35** *- оптимальный показатель*
!rm -rf ../colorized_photo.jpg
from google.colab import files
render_factor = 35  #@param {type: "slider", min: 5, max: 44}
image_path = colorizer.plot_transformed_image(path='../photo.jpg', render_factor=render_factor, compare=True)
!cp -r result_images/photo.jpg ../colorized_photo.jpg

#@markdown **Автоматическое скачивание:**
download_image = False #@param {type:"boolean"}
if download_image == True:
  files.download("result_images/photo.jpg")
else:
  show_image_in_notebook(image_path)

In [ ]:
#@title ###**Выход из раздела DeOldify** { display-mode: "form" }
#@markdown *Если вы собираетесь продолжить выполнять следующие пункты, то прежде всего обязательно выполните этот блок!*
!cp -r result_images/photo.jpg ../photo.jpg
%cd /content
clear_output()


### <b><font color="aquamarine" size="+1">2) 3D Ken Burns - Параллакс эффект</b>

Автор 3D Ken Burns: [Simon Niklaus](http://sniklaus.com/about/welcome)

*↓ Нажмите, чтобы открыть подраздел ↓*

In [ ]:
#@title ###**Клонируем репозиторий и качаем необходимые компоненты** { display-mode: "form" }
import os
import imageio
import numpy as np
import moviepy.editor as mpy
from moviepy.video.io.ffmpeg_writer import FFMPEG_VideoWriter
%matplotlib inline
%cd /content
os.environ['CUDA_HOME'] = "/usr/local/cuda"
!pip install moviepy
!pip install gevent
!sudo apt install python-gevent
!git clone https://github.com/sniklaus/3d-ken-burns.git 3D-Ken-Burns
%cd /content/3D-Ken-Burns
!mkdir 'results'
clear_output()

In [ ]:
#@title ###**Качаем предобученную модель** { display-mode: "form" }
!rm -rf models/disparity-estimation.pytorch
!rm -rf models/disparity-refinement.pytorch
!rm -rf models/pointcloud-inpainting.pytorch
try:
  !gdown https://drive.google.com/uc?id=1-0M3GAZkiOexs7gqgWuORzqYAp00DeE- -O ./models/disparity-estimation.pytorch
  !gdown https://drive.google.com/uc?id=1-11KRkOKxqLmKOQk69PDMYbaem-3mH04 -O ./models/disparity-refinement.pytorch
  !gdown https://drive.google.com/uc?id=1-1-VjUQHVDwxtLY9xn65c3LpKCeIW6jH -O ./models/pointcloud-inpainting.pytorch
except BaseException:
  !wget -O ./3d-ken-burns/models/disparity-estimation.pytorch http://content.sniklaus.com/kenburns/network-disparity.pytorch
  !wget -O ./3d-ken-burns/models/disparity-refinement.pytorch http://content.sniklaus.com/kenburns/network-refinement.pytorch
  !wget -O ./3d-ken-burns/models/pointcloud-inpainting.pytorch http://content.sniklaus.com/kenburns/network-inpainting.pytorch
clear_output()

In [ ]:
#@title ###**Создаем Параллакс эффект** { display-mode: "form" }
!cp -r  ../photo.jpg ./images/photo.jpg
!rm images/README.md
!rm images/doublestrike.jpg
!for image in ./images/*; do python autozoom.py --in $image --out ./results/$(basename $image | cut -f1 -d '.').mp4; done
clear_output()
#@markdown *Чтобы сохранить видео нажмите по нему правой кнопкой мыши и выберите* **Сохранить**
display(mpy.ipython_display("results/photo.mp4", height=400, autoplay=1, loop=1))

In [ ]:
#@title ###**Выход из раздела 3D Ken Burns** { display-mode: "form" }
#@markdown *Если вы собираетесь продолжить выполнять следующие пункты, то прежде всего обязательно выполните этот блок!*
!cp -r results/photo.mp4 ../parallax_video.mp4
!cp -r results/photo.mp4 ../video.mp4
%cd /content
clear_output()

---
## <b><font color="grey" size="+2">Бонус. Синтезируем аудио с помощью голоса ([DDSP](https://github.com/magenta/ddsp))
*↓ Нажмите, чтобы открыть раздел ↓*

Автор: [Magenta](https://github.com/magenta)

In [ ]:
#@title ###**Устанавливаем все необходимые библиотеки** { display-mode: "form" }
%cd /content
%tensorflow_version 2.x
!pip install -qU ddsp

from IPython.display import clear_output
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import warnings
warnings.filterwarnings("ignore")

import copy
import os
import time
import crepe
import ddsp
import ddsp.training
from ddsp.colab.colab_utils import (download, play, record, specplot, upload,
                                    DEFAULT_SAMPLE_RATE)
import gin
from google.colab import files
import librosa
import matplotlib.pyplot as plt
import numpy as np
import tensorflow.compat.v2 as tf
import tensorflow_datasets as tfds

sample_rate = DEFAULT_SAMPLE_RATE  # 16000
clear_output()
print('Готово!')

In [ ]:
#@title ###**Запишите или загрузите (.wav или .mp3) аудио** { display-mode: "form" }

#@markdown * Внимание! Аудио должно быть монотонным (*один голос или один инструмент*)
#@markdown * Если вы выбрали "**Record**", то запись начнется сразу же, как появится надпись "**Starting recording for X seconds...**"
#@markdown * Тестирование звукозаписи проводилось в **Google Chrome** и **Opera**. На других эта функция может не работать."

record_or_upload = "Record"  #@param ["Record", "Upload (.mp3 or .wav)"]

record_seconds =   10  #@param {type:"number", min:1, max:10, step:1}

if record_or_upload == "Record":
  audio = record(seconds=record_seconds)
else:
  filenames, audios = upload()
  audio = audios[0]
audio = audio[np.newaxis, :]
clear_output()

specplot(audio)
play(audio)

ddsp.spectral_ops.reset_crepe()

start_time = time.time()
audio_features = ddsp.training.eval_util.compute_audio_features(audio)
audio_features['loudness_db'] = audio_features['loudness_db'].astype(np.float32)
audio_features_mod = None

In [ ]:
#@title ###**Выберите инструмент** { display-mode: "form" }

#@markdown * **Violin** (Скрипка)

#@markdown * **Flute** (Флейта)

#@markdown * **Trumpet** (Труба)

#@markdown * **Tenor_Saxophone** (Тенор саксофон)

#@markdown * или загрузите свою собственную модель

#@markdown ---

model = 'Violin' #@param ['Violin', 'Flute', 'Flute2', 'Trumpet', 'Tenor_Saxophone','Upload your own (checkpoint folder as .zip)']
MODEL = model
#@markdown ---
GCS_CKPT_DIR = 'gs://ddsp/models/tf2'

def find_model_dir(dir_name):
  # Iterate through directories until model directory is found
  for root, dirs, filenames in os.walk(dir_name):
    for filename in filenames:
      if filename.endswith(".gin") and not filename.startswith("."):
        model_dir = root
        break
  return model_dir 


if model in ('Violin', 'Flute', 'Flute2', 'Trumpet', 'Tenor_Saxophone'):
  # Pretrained models.
  PRETRAINED_DIR = '/content/pretrained'
  # Copy over from gs:// for faster loading.
  !rm -r $PRETRAINED_DIR &> /dev/null
  !mkdir $PRETRAINED_DIR &> /dev/null
  model_dir = os.path.join(GCS_CKPT_DIR, 'solo_%s_ckpt' % model.lower())
  !gsutil cp $model_dir/* $PRETRAINED_DIR &> /dev/null
  model_dir = PRETRAINED_DIR
  gin_file = os.path.join(model_dir, 'operative_config-0.gin')

else:
  # User models.
  UPLOAD_DIR = '/content/uploaded'
  !mkdir $UPLOAD_DIR
  uploaded_files = files.upload()

  for fnames in uploaded_files.keys():
    print("Unzipping... {}".format(fnames))
    !unzip -o "/content/$fnames" -d $UPLOAD_DIR &> /dev/null
  model_dir = find_model_dir(UPLOAD_DIR)
  gin_file = os.path.join(model_dir, 'operative_config-0.gin')

# Parse gin config,
with gin.unlock_config():
  gin.parse_config_file(gin_file, skip_unknown=True)

# Assumes only one checkpoint in the folder, 'ckpt-[iter]`.
ckpt_files = [f for f in tf.io.gfile.listdir(model_dir) if 'ckpt' in f]
ckpt_name = ckpt_files[0].split('.')[0]
ckpt = os.path.join(model_dir, ckpt_name)

# Ensure dimensions and sampling rates are equal
time_steps_train = gin.query_parameter('DefaultPreprocessor.time_steps')
n_samples_train = gin.query_parameter('Additive.n_samples')
hop_size = int(n_samples_train / time_steps_train)

time_steps = int(audio.shape[1] / hop_size)
n_samples = time_steps * hop_size

# print("===Trained model===")
# print("Time Steps", time_steps_train)
# print("Samples", n_samples_train)
# print("Hop Size", hop_size)
# print("\n===Resynthesis===")
# print("Time Steps", time_steps)
# print("Samples", n_samples)
# print('')

gin_params = [
    'Additive.n_samples = {}'.format(n_samples),
    'FilteredNoise.n_samples = {}'.format(n_samples),
    'DefaultPreprocessor.time_steps = {}'.format(time_steps),
]

with gin.unlock_config():
  gin.parse_config(gin_params)


# Trim all input vectors to correct lengths 
for key in ['f0_hz', 'f0_confidence', 'loudness_db']:
  audio_features[key] = audio_features[key][:time_steps]
audio_features['audio'] = audio_features['audio'][:, :n_samples]


# Set up the model just to predict audio given new conditioning
model = ddsp.training.models.Autoencoder()
model.restore(ckpt)

# Build model by running a batch through it.
start_time = time.time()
_ = model(audio_features, training=False)

In [ ]:
#@title ###**Модификация и синтез аудио** { display-mode: "form" }

#@markdown Автокорректировка средней громкости, частоты и высоты тона (данная опция не всегда дает хороший результат):

auto_adjust = True #@param{type:"boolean"}


#@markdown *Контроль октавы (звук более глухой или звонкий):*
f0_octave_shift =  0 #@param {type:"slider", min:-2, max:2, step:1}
#@markdown *Контроль доверительного интервала звучания:*
f0_confidence_threshold =  0 #@param {type:"slider", min:0.0, max:1.0, step:0.05}
#@markdown *Контроль уровня громкости:*
loudness_db_shift = 0 #@param {type:"slider", min:-20, max:20, step:1}

#@markdown Экспериментируя с бегунками вы можете добиваться более реалистичного звучания

audio_features_mod = {k: v.copy() for k, v in audio_features.items()}


## Helper functions.
def shift_ld(audio_features, ld_shift=0.0):
  """Shift loudness by a number of ocatves."""
  audio_features['loudness_db'] += ld_shift
  return audio_features


def shift_f0(audio_features, f0_octave_shift=0.0):
  """Shift f0 by a number of ocatves."""
  audio_features['f0_hz'] *= 2.0 ** (f0_octave_shift)
  audio_features['f0_hz'] = np.clip(audio_features['f0_hz'], 
                                    0.0, 
                                    librosa.midi_to_hz(110.0))
  return audio_features


def mask_by_confidence(audio_features, confidence_level=0.1):
  """For the violin model, the masking causes fast dips in loudness. 
  This quick transient is interpreted by the model as the "plunk" sound.
  """
  mask_idx = audio_features['f0_confidence'] < confidence_level
  audio_features['f0_hz'][mask_idx] = 0.0
  # audio_features['loudness_db'][mask_idx] = -ddsp.spectral_ops.LD_RANGE
  return audio_features


def smooth_loudness(audio_features, filter_size=3):
  """Smooth loudness with a box filter."""
  smoothing_filter = np.ones([filter_size]) / float(filter_size)
  audio_features['loudness_db'] = np.convolve(audio_features['loudness_db'], 
                                           smoothing_filter, 
                                           mode='same')
  return audio_features

if auto_adjust:
  if MODEL in ['Violin', 'Flute', 'Flute2', 'Trumpet', 'Saxophone', 'Tenor_Saxophone']:
    # Adjust the peak loudness.
    l = audio_features['loudness_db']
    model_ld_avg_max = {
        'Violin': -34.0,
        'Flute': -45.0,
        'Flute2': -44.0,
        'Trumpet': -52.3,
        'Tenor_Saxophone': -31.2
    }[MODEL]
    ld_max = np.max(audio_features['loudness_db'])
    ld_diff_max = model_ld_avg_max - ld_max
    audio_features_mod = shift_ld(audio_features_mod, ld_diff_max)

    # Further adjust the average loudness above a threshold.
    l = audio_features_mod['loudness_db']
    model_ld_mean = {
        'Violin': -44.0,
        'Flute': -51.0,
        'Flute2': -53.0,
        'Trumpet': -69.2,
        'Tenor_Saxophone': -50.8
    }[MODEL]
    ld_thresh = -70.0
    ld_mean = np.mean(l[l > ld_thresh])
    ld_diff_mean = model_ld_mean - ld_mean
    audio_features_mod = shift_ld(audio_features_mod, ld_diff_mean)

    # Shift the pitch register.
    model_p_mean = {
        'Violin': 73.0,
        'Flute': 81.0,
        'Flute2': 74.0,
        'Trumpet': 65.8,
        'Tenor_Saxophone': 57.8
    }[MODEL]
    p = librosa.hz_to_midi(audio_features['f0_hz'])
    p[p == -np.inf] = 0.0
    p_mean = p[l > ld_thresh].mean()
    p_diff = model_p_mean - p_mean
    p_diff_octave = p_diff / 12.0
    round_fn = np.floor if p_diff_octave > 1.5 else np.ceil
    p_diff_octave = round_fn(p_diff_octave)
    audio_features_mod = shift_f0(audio_features_mod, p_diff_octave)

  else:
    print('\nUser uploaded model: disabling auto-adjust.')

  
audio_features_mod = shift_ld(audio_features_mod, loudness_db_shift)
audio_features_mod = shift_f0(audio_features_mod, f0_octave_shift)
audio_features_mod = mask_by_confidence(audio_features_mod, f0_confidence_threshold)

# Resynthesize Audio.
af = audio_features if audio_features_mod is None else audio_features_mod

# Run a batch of predictions.
start_time = time.time()
audio_gen = model(af, training=False)

print('----------------------------------------------------------------------------------------------------')
print('- Для скачивания аудиозаписи нажмите правой кнопкой мыши на плеер и выберите "Сохранить аудио как" -')
print('----------------------------------------------------------------------------------------------------')
print('\n')

# Audio.
print('Resynthesis')
play(audio_gen)

print('Original')
play(audio)

# Plot
specplot(audio_gen)
plt.title("Resynthesis")

specplot(audio)
_ = plt.title("Original")